In [1]:
import os
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI'

In [3]:
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    save_dir: Path
    save_path: Path

In [5]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_present_dir,
            data_path=config.data_present_path,
            save_dir=config.root_dir,
            save_path=config.data_path
            # tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [7]:
import string,time
from nltk.corpus import stopwords
import nltk
import emoji
from textblob import TextBlob
from datasets import load_dataset, load_from_disk
import pandas as pd
import sklearn
nltk.download('stopwords')

c:\Users\akish\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-01 23:39:59,392: INFO: config: TensorFlow version 2.12.0 available.]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def removePuntuations(self,text) -> str:
        punctuation = string.punctuation
        return text.translate(str.maketrans('', '', punctuation))

    def removeStopwords(self,text) -> str:
        newText = []

        for word in text.split():
            if word in stopwords.words('english'):
                newText.append('')
            else:
                newText.append(word)
        x = newText[:]
        newText.clear()
        return " ".join(x)


    def lemmatizeText(self,text) -> str:
        w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
        lemmatizer = nltk.stem.WordNetLemmatizer()
        sentence = ""
        for word in w_tokenizer.tokenize(text):
            sentence = sentence + lemmatizer.lemmatize(word) + " "
        return sentence

    def chatConversion(self,text) -> str:
        newText = []
        for word in text.split():
            if word.upper() in chatWords:
                newText.append(chatWords[word.upper()])
            else:
                newText.append(word)
        return " ".join(newText)

    def decodeEmoji(self,text) -> str:
        return emoji.demojize(text).replace(":",'').replace("_"," ")

    def correctText(self,text) -> str:
        return TextBlob(text).correct().string

    def labelling(self,sentiment):
        if sentiment == 'positive':
            return 1
        elif sentiment == 'negative':
            return 0
        elif sentiment == 1:
            return 1
        elif sentiment == 0:
            return 0

    def shuffle(self,sentiData):
        for i in range(50):
            sentiData = sklearn.utils.shuffle(sentiData)
        return sentiData

    def load(self):
        sentiData = pd.read_excel(self.config.data_path)
        return sentiData
    
    def saveToExcel(self,sentiData):
            return sentiData.to_excel(os.path.join(self.config.save_path))
        


In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    print(data_transformation.config.data_path)
    sentiData = data_transformation.load()
    print(sentiData.head())
    data_transformation.saveToExcel(sentiData)
    # sentiData = data_transformation.shuffle(data)
    # print(sentiData.head())
    # sentiData['sentiment'] = sentiData['sentiment'].apply(data_transformation.labelling)
    # print(sentiData.head())
    # # sentiData['Review'] = sentiData['Review'].apply(data_transformation.removePuntuations)
    # # print("------------------handling punctuations done---------------------")
    # print(sentiData.head())
    # sentiData['Review'] = sentiData['Review'].apply(data_transformation.removeStopwords)
    # print("---------------------removing stopwords done---------------------")
    # print(sentiData.head())
    # sentiData['Review'] = sentiData['Review'].apply(data_transformation.lemmatizeText)
    # print("--------------------lemmatizing done-----------------------")
    # print(sentiData.head())
    # data_transformation.saveToExcel(sentiData)
except Exception as e:
    raise e

[2024-01-01 23:46:10,993: INFO: common: yaml file: {yaml_file}]
[2024-01-01 23:46:10,995: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-01 23:46:10,997: INFO: common: yaml file: {yaml_file}]
[2024-01-01 23:46:10,998: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-01 23:46:10,999: INFO: common: created directory at: artifacts]
[2024-01-01 23:46:11,000: INFO: common: created directory at: artifacts/data_transformation/sentimentDataset]
artifacts/data_ingestion/SentimentDataFinal.xlsx
   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                              Review sentiment  
0  If you decide to eat here, just be aware it is...  positive  
1  I've taken a lot of spin classes over the year...  positive  
2  Family diner. Had the buffet. Eclectic assortm...  positive  
3  Wow!  Yummy, di

In [ ]:
from datasets import load_dataset, load_from_disk
def load(self):
    b = load_dataset(self.config.data_path)

In [8]:
import os
import json
import gzip
import nltk
import pandas as pd
from urllib.request import urlopen
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import keras
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import tensorflow as tf
from tensorflow import keras as ks
import time

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentimentData['Review'], encoded_labels, stratify = encoded_labels)

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
# padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)